# Deploy the Model using Gradio to a HF Space

In [ ]:
# clone the hugging face space
!git clone https://huggingface.co/spaces/BrandonFors/PlantDiseaseDetection

Cloning into 'PlantDiseaseDetection'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 66 (delta 6), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (66/66), 590.11 KiB | 10.54 MiB/s, done.


In [ ]:
%cd PlantDiseaseDetection/
!git clone https://huggingface.co/BrandonFors/effnetv2_s_plant_disease

/content/PlantDiseaseDetection
Cloning into 'effnetv2_s_plant_disease'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 13), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (60/60), 12.05 KiB | 771.00 KiB/s, done.
Filtering content: 100% (28/28), 233.83 MiB | 37.61 MiB/s, done.


In [ ]:
!cp effnetv2_s_plant_disease/modeling.py .
!cp effnetv2_s_plant_disease/configuration.py .

In [ ]:
!rm -r ./effnetv2_s_plant_disease

In [ ]:
# unzip the examples.zip (manually imported)
from pathlib import Path
import requests
import os
import zipfile

examples_path = Path("examples/")
examples_path.mkdir(exist_ok=True, parents=True)

with zipfile.ZipFile("examples.zip", "r") as zip_ref:
  print("Unzipping utils")
  zip_ref.extractall(examples_path)

# Remove .zip file
os.remove("examples.zip")

Unzipping utils


## Create an app.py script to run the model
Images were manually downloaded from test split


In [ ]:
# %%writefile app.py
import gradio as gr
import os
import torch
import torchvision
from modeling import EffNetPlantDiseaseClassification
from timeit import default_timer as timer
from typing import Dict, Tuple
# load the model from hugging face repo
model = EffNetPlantDiseaseClassification.from_pretrained("BrandonFors/effnetv2_s_plant_disease")

# load transforms for effnetv2-s from torchvision
effnetv2_s_weights = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT
effnetv2_s_auto_transforms = effnetv2_s_weights.transforms()

# define class names
class_names = ['Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  'Apple___healthy',
  'Blueberry___healthy',
  'Cherry_(including_sour)___Powdery_mildew',
  'Cherry_(including_sour)___healthy',
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
  'Corn_(maize)___Common_rust_',
  'Corn_(maize)___Northern_Leaf_Blight',
  'Corn_(maize)___healthy',
  'Grape___Black_rot',
  'Grape___Esca_(Black_Measles)',
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
  'Grape___healthy',
  'Orange___Haunglongbing_(Citrus_greening)',
  'Peach___Bacterial_spot',
  'Peach___healthy',
  'Pepper,_bell___Bacterial_spot',
  'Pepper,_bell___healthy',
  'Potato___Early_blight',
  'Potato___Late_blight',
  'Potato___healthy',
  'Raspberry___healthy',
  'Soybean___healthy',
  'Squash___Powdery_mildew',
  'Strawberry___Leaf_scorch',
  'Strawberry___healthy',
  'Tomato___Bacterial_spot',
  'Tomato___Early_blight',
  'Tomato___Late_blight',
  'Tomato___Leaf_Mold',
  'Tomato___Septoria_leaf_spot',
  'Tomato___Spider_mites Two-spotted_spider_mite',
  'Tomato___Target_Spot',
  'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
  'Tomato___Tomato_mosaic_virus',
  'Tomato___healthy']
# create predict function
def predict(img):
  # start the timer to time the prediction
  start_time = timer()
  # Transform the input image for use
  img = effnetv2_s_auto_transforms(img).unsqueeze(0)
  # Put the model in eval mode to make prediction
  model.eval()
  with torch.inference_mode():
    pred_logits = model(img)["logits"]
    pred_probs = torch.softmax(pred_logits, dim=1)
  # create a prediction label and prediction probabilities dict
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # calc pred time
  pred_time = round(timer() - start_time, 4)
  # return the pred dict and time
  return pred_labels_and_probs, pred_time

# Gradio App

# Create title, description, and article
title = "EffNetV2_S_PlantDisease"
description = "EffNetV2-S trained on Plant Village Dataset"
article = "Personal project"

# Create example list
example_list = [["examples/" + example] for example in os.listdir("examples")]

# Create the Gradio Demo
demo = gr.Interface(fn=predict, # maps inputs to outputs
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)
# launch the demo
demo.launch(debug=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/81.6M [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df31a0bb991149f0f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from PIL import Image
# test the model predict function on an image
img = Image.open("/content/PlantDiseaseDetection/examples/Tomato___Tomato_mosaic_virus.JPG")
predict(img)

({'Apple___Apple_scab': 1.3920645081100247e-08,
  'Apple___Black_rot': 7.18815629241476e-09,
  'Apple___Cedar_apple_rust': 2.0681770962482915e-08,
  'Apple___healthy': 1.0387409821177584e-09,
  'Blueberry___healthy': 1.367214395031624e-07,
  'Cherry_(including_sour)___Powdery_mildew': 1.4164062349664164e-06,
  'Cherry_(including_sour)___healthy': 7.191930961880644e-08,
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 5.695522942161801e-10,
  'Corn_(maize)___Common_rust_': 1.76571812726678e-09,
  'Corn_(maize)___Northern_Leaf_Blight': 5.823508897329077e-10,
  'Corn_(maize)___healthy': 2.050324798830161e-09,
  'Grape___Black_rot': 4.1975294351459524e-08,
  'Grape___Esca_(Black_Measles)': 5.167630590108274e-08,
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 2.4364295470036268e-08,
  'Grape___healthy': 1.5681215215224142e-09,
  'Orange___Haunglongbing_(Citrus_greening)': 3.0046074517287025e-09,
  'Peach___Bacterial_spot': 1.3287447808707498e-09,
  'Peach___healthy': 3.7960794507796

In [ ]:
%%writefile requirements.txt
torch
torchvision
gradio
transformers

Overwriting requirements.txt


In [ ]:
import torch

## Upload to hugging face

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!git config --global user.email ""
!git config --global user.name ""

In [ ]:
!git add *

In [ ]:
!git commit -m "uploading files to space"

[main 4f09bb0] uploading files to space
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git push

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 303 bytes | 303.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/spaces/BrandonFors/PlantDiseaseDetection
   11dc4a2..4f09bb0  main -> main
